#**Qual a finalidade desse notebook?**
Este nootebook tem por fim fazer alguns ajustes no **dataset** (o construído na etapa anterior), conforme interesse da pesquisa relacionada à compreensão de como certa configuração sócio-econômica favorece, ou não, a realização dos nívies de contanminação e/ou óbitos da relacionados à Covid19

#**Hands-on ...**

## **1. Carregar Libs, Carregar *dataset*, realizar ajustes & exportar**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# get libs

!pip install --upgrade scikit-learn
!pip install --upgrade pivottablejs

from time import sleep
import zipfile
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import plot_tree
import joblib

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

# Rich visual representation of estimators (new 0.23.2)
from sklearn import set_config
set_config(display='diagram')

     |████████████████████████████████| 6.8MB 5.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [29]:
### definir qual dataset gerar
ds_src = 'dataset_ex'
ds_dst = 'dataset_ex_02'

In [30]:
#######################################
## Descompactando o arquivo do dataset
#######################################

DIR_BASE = '/content/drive/MyDrive/Colab Notebooks/COVID-19/Databases'

with zipfile.ZipFile(DIR_BASE+'/'+ds_src+'.zip','r') as zip_ref:
  zip_ref.extractall(DIR_BASE+'/')

# Globals
def isNaN(value):
  return isinstance(value, float) and math.isnan(value)

In [31]:
#########################
## Carregar o dataset
#########################

df = pd.read_csv(DIR_BASE+'/'+ds_src+'.csv', sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4643 entries, 0 to 4642
Columns: 113 entries, IBGE to MEDU52
dtypes: float64(3), int64(17), object(93)
memory usage: 4.0+ MB


In [32]:
df.drop(columns=['data', 'JAN','FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'], inplace=True)

In [29]:
df.head()

,IBGE,PMATPUB_EF,PMATPUB_EM,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,TXMOINF,TXBRUTAMORT,TXCOBPLP,PINTERSAP,PINTERDRSAI,REN_TRPCBF_D,REN_TRPCBPC_D,PDEFAGUA,PDEFESGOTO,PDEFLIXO,PDEFSAN,PPOB_POS,PBF,SNIS_PAGUA,PFOCOS,POP_TOT,estado,municipio,casosAcumulado,obitosAcumulado,interior/metropolitana,Região,Porte,Capital,LON,LAT,ANNUAL,MTIP03,MTIP05,MTIP07,MTIP09,...,MSAU28,MSAU29,MSAU2911,MSAU2914,MSAU30,MSAU3011,MSAU31,MSAU33,MSAU34,MSAU42,MSAU43,MSAU44,MSAU456,MSAU451,MSAU452,MSAU453,MSAU454,MSAU47,MSAU48,MSAU51,MEDU01,MEDU111,MEDU113,MEDU115,MEDU116,MEDU117,MEDU119,MEDU1110,MEDU1112,MEDU1113,MEDU1114,MEDU1115,MEDU1116,MEDU1117,MEDU1118,MEDU16,MEDU161b,MEDU21,MEDU48,MEDU52
0,110001,"100,00","100,00","19,90","25,10","6,2","4,8","58,1","61,7","14,12","5,39","6,01","41,53","4,64","79,30","142,56","80,81","93,09","53,62","50,15","84,71","66,31","73,27","0,52",25437,RO,Alta Floresta D'Oeste,903,11,0.0,Região Norte,Pequeno II,Interior,-61.9958,-11.9287,3387,Sim,Sim,Sim,Não,...,Não,Sim,5,75,Sim,1,Não,Sim,Sim,Sim,Não,Na equipe de Saúde da Família responsável pelo...,Sim,Sim,Não,Não,Não,Não,Sim,Sim,Secretaria exclusiva,Não,Sim,Não,Não,Não,Não,Sim,Sim,Não,Não,Não,Não,Não,Não,Sim,2001,Não,Secretário de educação,Sim
1,110002,"89,42","95,84","20,20","31,40","5,5","4,9","63,4","64,8","13,34","4,68","7,06","25,15","2,23","41,87","178,73","64,66","89,42","14,74","13,47","79,91","57,66","94,81","0,78",107345,RO,Ariquemes,5840,99,0.0,Região Norte,Médio,Interior,-63.0330,-9.9061,3067,Sim,Sim,Não,Não,...,Sim,Sim,18,122,Sim,6,Não,Sim,Sim,Sim,Não,Na equipe de Saúde da Família responsável pelo...,Sim,Sim,Não,Não,Não,Sim,Sim,Sim,Secretaria exclusiva,Sim,Sim,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Sim,2007,Sim,Secretário de educação,Sim
2,110003,"100,00","100,00","11,80","27,80","6,1","5,1","73,5","60,7","0,00","4,66","3,31","47,04","7,47","36,76","103,04","64,37","40,09","40,23","17,92","75,32","47,87","85,80","0,06",6224,RO,Cabixi,137,5,0.0,Região Norte,Pequeno I,Interior,-60.5446,-13.4951,3831,Sim,Sim,Sim,Não sabe informar,...,Não,Sim,3,23,Sim,1,Não,Sim,Sim,Sim,Sim,Outro,Sim,Não,Não,Não,Sim,Não,Sim,Sim,Secretaria em conjunto com outras políticas se...,Não,Não,Não,Não,Não,Não,Sim,Não,Não,Não,Sim,Não,Não,Não,Sim,2016,Não,Prefeito,Sim
3,110004,"82,46","80,47","14,00","28,80","6,3","5,1","60,8","54,2","9,64","5,66","9,07","23,99","2,05","29,62","156,91","26,36","56,03","22,47","20,70","76,88","47,86","100,00","0,18",88507,RO,Cacoal,2534,35,0.0,Região Norte,Médio,Interior,-61.4566,-11.4347,3374,Sim,Sim,Sim,Não sabe informar,...,Não,Sim,15,91,Sim,7,Não,Sim,Sim,Não,Não,Setor específico do órgão gestor municipal da ...,Sim,Sim,Sim,Não,Não,Não,Sim,Sim,Secretaria exclusiva,Sim,Sim,Não,Não,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Sim,2010,Não,Secretário de educação,Sim
4,110005,"96,31","97,47","15,50","26,90","6,4","5,3","65,8","39,2","3,61","4,52","3,81","37,16","6,46","44,39","172,70","68,13","91,35","11,01","10,22","68,43","46,38","64,95","0,49",17934,RO,Cerejeiras,276,6,0.0,Região Norte,Pequeno I,Interior,-60.8173,-13.1874,3744,Sim,Sim,Não,Não,...,Não,Sim,5,23,Sim,1,Não,Sim,Sim,Sim,Sim,Na equipe de Saúde da Família responsável pelo...,Não,NaN,NaN,NaN,NaN,Não,Sim,Sim,Secretaria exclusiva,Não,Sim,Não,Não,Não,Não,Sim,Não,Sim,Não,Não,Não,Não,Não,Sim,2011,Sim,Secretário de educação,Sim


In [33]:
#######################################
## Ajustes - Converter formato de numeros nas colunas
#######################################

def treat_comma(x):
  if (x.dtype == 'object' and (x.str.contains(',')).all()): 
    return x.str.replace(',','.')
  return x

df = (df.apply(treat_comma, axis = 0)).convert_dtypes()

In [34]:
numerical_columns = df.loc[:,'PMATPUB_EF':'POP_TOT'].columns.to_list()
numerical_columns.append('casosAcumulado')
numerical_columns.append('obitosAcumulado')
numerical_columns.append('ANNUAL')
numerical_columns

['PMATPUB_EF',
 'PMATPUB_EM',
 'DIST_EF_PUB',
 'DIST_EM_PUB',
 'IDEB_AI',
 'IDEB_AF',
 'DOCSUP_EF_PUB',
 'DOCSUP_EM_PUB',
 'TXMOINF',
 'TXBRUTAMORT',
 'TXCOBPLP',
 'PINTERSAP',
 'PINTERDRSAI',
 'REN_TRPCBF_D',
 'REN_TRPCBPC_D',
 'PDEFAGUA',
 'PDEFESGOTO',
 'PDEFLIXO',
 'PDEFSAN',
 'PPOB_POS',
 'PBF',
 'SNIS_PAGUA',
 'PFOCOS',
 'POP_TOT',
 'casosAcumulado',
 'obitosAcumulado',
 'ANNUAL']

In [35]:
##################
## Ajustes Gerais
##################

def transform_to_numeric(x):
  if (x.name in numerical_columns):
    return pd.to_numeric(x)
  return x

df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df.apply(transform_to_numeric)
df = df.rename(columns={'interior/metropolitana': 'reg_metro' })

df = df.astype({ 
  'casosAcumulado': float, 
  'obitosAcumulado': float, 
  'reg_metro': str, 
  'Capital': str, 
  'POP_TOT': float, 
  'ANNUAL': float 
})

df.loc[df['reg_metro'] == '0', 'reg_metro'] = 'Interior'
df.loc[df['reg_metro'] == '1', 'reg_metro'] = 'Região Metropolitana'

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3672 entries, 0 to 4642
Data columns (total 100 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IBGE             3672 non-null   Int64  
 1   PMATPUB_EF       3672 non-null   float64
 2   PMATPUB_EM       3672 non-null   float64
 3   DIST_EF_PUB      3672 non-null   float64
 4   DIST_EM_PUB      3672 non-null   float64
 5   IDEB_AI          3672 non-null   float64
 6   IDEB_AF          3672 non-null   float64
 7   DOCSUP_EF_PUB    3672 non-null   float64
 8   DOCSUP_EM_PUB    3672 non-null   float64
 9   TXMOINF          3672 non-null   float64
 10  TXBRUTAMORT      3672 non-null   float64
 11  TXCOBPLP         3672 non-null   float64
 12  PINTERSAP        3672 non-null   float64
 13  PINTERDRSAI      3672 non-null   float64
 14  REN_TRPCBF_D     3672 non-null   float64
 15  REN_TRPCBPC_D    3672 non-null   float64
 16  PDEFAGUA         3672 non-null   float64
 17  PDEFESGOTO   

In [36]:
###################################
## Exportar o resultado de dataset 
###################################

compression_opts = dict(method='zip', archive_name=ds_dst+'.csv')  
df.to_csv (DIR_BASE+'/'+ds_dst+'.zip', sep=';', index=False, header=True, compression=compression_opts)